In [1]:
import sys
sys.path.append("..")

import torch
import torch.optim as optim

import matplotlib.pyplot as plt

In [2]:
from datasets import *
from models.ad_models import *

In [3]:
# dataset = MVTecDataset("bottle")
dataloader = get_ad_dataloader("mvtec", "vae", batch_size = 12, category="bottle")

In [4]:
for batch in dataloader:
    break

In [5]:
batch[0].shape

torch.Size([12, 3, 256, 256])

In [6]:
model = VaeADModel()

In [7]:
batch[0].shape

torch.Size([12, 3, 256, 256])

In [8]:
out = model(batch[0])

In [18]:
optimizer = optim.AdamW(model.parameters(), lr=0.1)
sched1 = optim.lr_scheduler.LinearLR(optimizer, start_factor=0.1, end_factor=1.0, total_iters=5)
sched2 = optim.lr_scheduler.LinearLR(optimizer, start_factor=1.0, end_factor=0.1, total_iters=45)
sched = optim.lr_scheduler.SequentialLR(optimizer, schedulers=[sched1, sched2], milestones=[5])

In [19]:
sched.get_last_lr

<bound method LRScheduler.get_last_lr of <torch.optim.lr_scheduler.SequentialLR object at 0x7ffaf810a850>>

In [20]:
for i in range(50):
    print(i, sched.get_last_lr())
    sched.step()

0 [0.010000000000000002]
1 [0.028000000000000004]
2 [0.046000000000000006]
3 [0.064]
4 [0.082]
5 [0.1]
6 [0.098]
7 [0.096]
8 [0.094]
9 [0.092]
10 [0.09]
11 [0.088]
12 [0.086]
13 [0.08399999999999999]
14 [0.08199999999999999]
15 [0.07999999999999999]
16 [0.07799999999999999]
17 [0.07599999999999998]
18 [0.07399999999999998]
19 [0.07199999999999998]
20 [0.06999999999999998]
21 [0.06799999999999998]
22 [0.06599999999999998]
23 [0.06399999999999997]
24 [0.06199999999999997]
25 [0.05999999999999998]
26 [0.057999999999999975]
27 [0.05599999999999998]
28 [0.05399999999999998]
29 [0.051999999999999984]
30 [0.04999999999999999]
31 [0.04799999999999999]
32 [0.04599999999999999]
33 [0.04399999999999999]
34 [0.041999999999999996]
35 [0.039999999999999994]
36 [0.03799999999999999]
37 [0.03599999999999999]
38 [0.03399999999999999]
39 [0.03199999999999999]
40 [0.02999999999999999]
41 [0.02799999999999999]
42 [0.025999999999999992]
43 [0.02399999999999999]
44 [0.02199999999999999]
45 [0.01999999999999

In [22]:
sched.get_last_lr()

[0.009999999999999992]